In [1]:
import sys
import gc
import wandb

import pandas as pd
from sklearn.model_selection import StratifiedKFold
import numpy as np
from sklearn.metrics import roc_auc_score
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)

from datasets import Dataset
from tqdm.auto import tqdm
from transformers import PreTrainedTokenizerFast

from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import VotingClassifier
import os
os.environ["TOKENIZERS_PARALLELISM"] = "False"

In [2]:
import sklearn
import lightgbm

print(sklearn.__version__)
print(lightgbm.__version__)

## log into wandb
wandb.login()

1.2.2
3.3.2


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: peng_sun. Use `wandb login --relogin` to force relogin


True

In [3]:
test = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/test_essays.csv')
sub = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/sample_submission.csv')
org_train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/train_essays.csv')

In [4]:
train = pd.read_csv('/home/peng_sun2/s3shared/kaggle/llm-2023/data/daigt/train_v2_drcat_02.csv', sep=',')
display(train.shape);

(44868, 5)

#### Read and append the generated data

In [5]:
train_gen_data = pd.read_parquet('/home/peng_sun2/s3shared/kaggle/llm-2023/external_data/gen_data_21122023.parquet')
display(train_gen_data.shape)

(18894, 5)

In [6]:
train = train.drop_duplicates(subset = ['text'])
train.reset_index(drop = True, inplace = True)

### append the generated data
# train = pd.concat([train, train_gen_data], axis = 0).reset_index(drop = True)
# display(train.shape)

In [7]:
train_0 = train.loc[train['label'] ==0, :].reset_index(drop = True)
train_1 = train.loc[train['label'] ==1, :].reset_index(drop = True)

In [8]:
train_0.shape

(27371, 5)

In [9]:
train_1.shape

(17497, 5)

In [10]:
# min_length = min(len(train_0), len(train_1))
# min_length

In [11]:
# sorted_train= pd.DataFrame()
# for i in tqdm(range(min_length), total = min_length):
#     sorted_train = pd.concat([sorted_train, pd.DataFrame(train_0.iloc[i,:]).T], axis = 0)
#     sorted_train = pd.concat([sorted_train, pd.DataFrame(train_1.iloc[i,:]).T], axis = 0)

In [12]:
# if len(train_0) > min_length:
#     sorted_train = pd.concat([sorted_train, train_0.iloc[min_length:, :]], axis = 0)
# elif len(train_1) > min_length:
#     sorted_train = pd.concat([sorted_train, train_1.iloc[min_length:, :]], axis = 0)

In [13]:
from tqdm import tqdm 
def sort_train(train):
    train_0 = train.loc[train['label'] ==0, :].reset_index(drop = True)
    train_1 = train.loc[train['label'] ==1, :].reset_index(drop = True)
    
    min_length = min(len(train_0), len(train_1))
    
    sorted_train= pd.DataFrame()
    for i in tqdm(range(min_length), total = min_length):
        sorted_train = pd.concat([sorted_train, pd.DataFrame(train_0.iloc[i,:]).T], axis = 0)
        sorted_train = pd.concat([sorted_train, pd.DataFrame(train_1.iloc[i,:]).T], axis = 0)
    
    if len(train_0) > min_length:
        sorted_train = pd.concat([sorted_train, train_0.iloc[min_length:, :]], axis = 0)
    elif len(train_1) > min_length:
        sorted_train = pd.concat([sorted_train, train_1.iloc[min_length:, :]], axis = 0)
        
    del train_0, train_1, train
    sorted_train['label'] = sorted_train['label'].astype(np.int32)
    return sorted_train.reset_index(drop = True)

In [14]:
display(train.shape)
train.head(10)

(44868, 5)

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
2,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
3,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
4,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
5,Cell phone use should not be legal while drivi...,0,Phones and driving,persuade_corpus,False
6,Phones and Driving\n\nDriving is a good way to...,0,Phones and driving,persuade_corpus,False
7,PHONES AND DRIVING\n\nIn this world in which w...,0,Phones and driving,persuade_corpus,False
8,People are debating whether if drivers should ...,0,Phones and driving,persuade_corpus,False
9,Texting and driving\n\nOver half of drivers in...,0,Phones and driving,persuade_corpus,False


In [15]:
train = sort_train(train)

100%|██████████| 17497/17497 [00:45<00:00, 383.98it/s]


In [16]:
display(train.shape)
train.head(10)

(44868, 5)

,text,label,prompt_name,source,RDizzl3_seven
0,Phones\n\nModern humans today are always on th...,0,Phones and driving,persuade_corpus,False
1,"In recent years, technology has had a profoun...",1,Car-free cities,mistral7binstruct_v2,True
2,This essay will explain if drivers should or s...,0,Phones and driving,persuade_corpus,False
3,I strongly believe that meditation and mindful...,1,Distance learning,llama_70b_v1,False
4,Driving while the use of cellular devices\n\nT...,0,Phones and driving,persuade_corpus,False
5,One way school administrators can attempt to c...,1,Cell phones at school,chat_gpt_moth,False
6,Phones & Driving\n\nDrivers should not be able...,0,Phones and driving,persuade_corpus,False
7,While summer is meant as a break from the regu...,1,Summer projects,darragh_claude_v7,False
8,Cell Phone Operation While Driving\n\nThe abil...,0,Phones and driving,persuade_corpus,False
9,The use of Facial Action Coding System (FACS) ...,1,Facial action coding system,darragh_claude_v6,True


In [17]:
LOWERCASE = False
VOCAB_SIZE = 30522

In [18]:
## Create byte-pair encoding tokenizer
raw_tokenizer = Tokenizer(models.BPE(unk_token='[UNK]'))

In [19]:
## adding normalization and pre_tokenizer
raw_tokenizer.normalizer = normalizers.Sequence([normalizers.NFC()]+ [normalizers.Lowercase()] if LOWERCASE else [])
raw_tokenizer.pre_tokenizer = pre_tokenizers.ByteLevel()

In [20]:
# Adding special tokens and creating trainer instance
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.BpeTrainer(vocab_size = VOCAB_SIZE, special_tokens = special_tokens)

In [21]:
# creating huggingface dataset object
dataset = Dataset.from_pandas(test[['text']])
def train_corp_iter():
    for i in range(0, len(dataset), 1000):
        yield dataset[i:i+1000]['text']
raw_tokenizer.train_from_iterator(train_corp_iter(), trainer = trainer)
tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=raw_tokenizer,
    unk_token="[UNK]",
    pad_token="[PAD]",
    cls_token="[CLS]",
    sep_token="[SEP]",
    mask_token="[MASK]",
)
tokenized_texts_test = []
for text in tqdm(test['text'].tolist()):
    tokenized_texts_test.append(tokenizer.tokenize(text))

tokenized_texts_train = []

for text in tqdm(train['text'].tolist()):
    tokenized_texts_train.append(tokenizer.tokenize(text))

/home/ubuntu/anaconda3/lib/python3.9/site-packages/pyarrow/pandas_compat.py:358: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


100%|██████████| 44868/44868 [02:03<00:00, 362.21it/s]


In [22]:
tokenized_texts_test[1]

['ĠBbb', 'Ġccc', 'Ġddd', '.']

In [23]:
def dummy(text):
    return text

In [24]:
vectorizer = TfidfVectorizer(ngram_range = (3, 5), lowercase=False, sublinear_tf = True, analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None,
                            strip_accents = 'unicode',
                            dtype = np.float32)
vectorizer.fit(tokenized_texts_test)

# Getting vocab
vocab = vectorizer.vocabulary_

print(vocab)

vectorizer = TfidfVectorizer(ngram_range=(3, 5), lowercase=False, sublinear_tf=True, 
                             #vocabulary=vocab,
                            analyzer = 'word',
                            tokenizer = dummy,
                            preprocessor = dummy,
                            token_pattern = None, strip_accents='unicode',
                            dtype = np.float32
                            )

tf_train = vectorizer.fit_transform(tokenized_texts_train)
tf_test = vectorizer.transform(tokenized_texts_test)
## get the vocabulary of the training dataset
train_vocab = vectorizer.vocabulary_

del vectorizer
gc.collect()

{'ĠAaa Ġbbb Ġccc': 0, 'Ġbbb Ġccc .': 6, 'ĠAaa Ġbbb Ġccc .': 1, 'ĠBbb Ġccc Ġddd': 2, 'Ġccc Ġddd .': 7, 'ĠBbb Ġccc Ġddd .': 3, 'ĠCCC Ġddd Ġeee': 4, 'Ġddd Ġeee .': 8, 'ĠCCC Ġddd Ġeee .': 5}


0

### Convert the type from numpy.float64 to numpy.float16

In [25]:
# import numpy as np
# from scipy.sparse import csr_matrix

# ## convert data for tf_train
# data  = tf_train.data.astype(np.float16)
# indices = tf_train.indices
# indptr = tf_train.indptr
# tf_train = csr_matrix((data, indices, indptr), shape = tf_train.shape)


# ## convert data for tf_test
# data = tf_test.data.astype(np.float16)
# indices = tf_test.indices
# indptr = tf_test.indptr
# tf_test = csr_matrix((data, indices, indptr), shape = tf_test.shape)

# del data, indices, indptr
# gc.collect()

## Start training

In [26]:
# ### save train_vocab
# import joblib
# model_path = '/home/peng_sun2/s3shared/kaggle/llm-2023/baseline1/'
# joblib.dump(train_vocab, f'{model_path}vocab_train.pkl')

In [27]:


# # Check if it's in scoring stage
# if len(test.text.values) <= 5:
#     # if not, just sample submission
#     sub.to_csv('submission.csv', index=False)
# else:
# otherwise, run fitting process

sweep_config = {
    'method': 'bayes',
    'metric': {'name': 'validatoin_auc', 'goal': 'maximize'},
    'parameters': {
        'max_depth': {'min': 3, 'max': 100},
        'min_child_weight': {'min': 1, 'max': 10},
        'gamma': {'min': 0, 'max': 10},
        'subsample': {'min': 0.5, 'max': 1.0},
        'colsample_bytree': {'min': 0.5, 'max': 1.0},
        'lambda_value': {'min': 0.0, 'max': 2.0},
        'alpha': {'min': 0.0, 'max': 2.0},
        'num_boost_round': {'min': 100, 'max': 2000}
    }
}

sweep_id = wandb.sweep(sweep_config, project="llm-2023-xgboost-hyperparameter")


Create sweep with ID: z97en3lz
Sweep URL: https://wandb.ai/peng_sun/llm-2023-xgboost-hyperparameter/sweeps/z97en3lz


In [28]:
def split_range_into_parts(start, end, num_parts):
    splits = np.linspace(start, end, num_parts + 1)
    ranges = [(int(np.floor(splits[i])), int(np.ceil(splits[i + 1]))) for i in range(num_parts)]
    return np.array(ranges)

In [29]:
def increment_train_catboost(tf_train, y_train, cat, num_parts):
    split_ranges = split_range_into_parts(0, tf_train.shape[0], num_parts);
    #print(split_ranges)
    
    ## incrementally train catboost
    for i, (start, end) in enumerate(split_ranges):
        print('start, end ', start, ' ', end)
        train_X_split = tf_train[start:end, :]
        train_y_split = y_train[start:end]
        
        #print(train_X_split.shape, train_y_split.shape)
        if i ==0:
            cat.fit(train_X_split, train_y_split)
        else: 
            cat.fit(train_X_split, train_y_split, init_model = cat)
        print(f'part {i} is finished ...')
    return cat
    

In [30]:
y_train = train['label'].values

In [31]:
import xgboost as xgb

## Test xgboost code

In [32]:
# from sklearn.model_selection import StratifiedGroupKFold
# from sklearn import metrics

# skf = StratifiedGroupKFold(n_splits = 5)
# skf.get_n_splits(tf_train, y_train, groups =train['prompt_name'])

# for i, (train_idx, test_idx) in enumerate(skf.split(tf_train, y_train, groups = train['prompt_name'])):
#     train_X = tf_train[train_idx]
#     eval_X = tf_train[test_idx];
#     train_y = y_train[train_idx]
#     eval_y = y_train[test_idx];
#     break;

In [33]:
# dtrain = xgb.DMatrix(train_X, label = train_y)
# deval = xgb.DMatrix(eval_X, label=eval_y)

In [34]:
# # Set up your training parameters
# param = {
#     'max_depth': 3,  # the maximum depth of each tree
#     'eta': 0.3,  # the training step for each iteration
#     'objective': 'binary:logistic',  # binary classification
#     'eval_metric': 'logloss'  # evaluation metric
# }
# num_round = 100  # the number of training iterations

# bst = xgb.train(
#     param,
#     dtrain,
#     num_round, 
#     evals = [(deval, 'eval')],
#     early_stopping_rounds=10
# )



In [35]:
# eval_preds = bst.predict(deval)
# fpr, tpr, thresholds = metrics.roc_curve(eval_y, eval_preds, pos_label = 1)
# eval_auc = metrics.auc(fpr, tpr)
# eval_auc

## Train

In [ ]:
from sklearn.model_selection import StratifiedGroupKFold
from sklearn import metrics
import lightgbm as lgb
import gc

## cv split
skf = StratifiedGroupKFold(n_splits = 5)
skf.get_n_splits(tf_train, y_train, groups =train['prompt_name'])

def train_sweep():
    with wandb.init() as run:
        config = wandb.config
        for i, (train_idx, test_idx) in enumerate(skf.split(tf_train, y_train, groups = train['prompt_name'])):
            train_X = tf_train[train_idx]
            eval_X = tf_train[test_idx];
            train_y = y_train[train_idx]
            eval_y = y_train[test_idx];
            
            dtrain = xgb.DMatrix(train_X, label = train_y)
            deval = xgb.DMatrix(eval_X, label=eval_y)
            params = {
                'max_depth': config.max_depth,
                'min_child_weight': config.min_child_weight,
                'gamma': config.gamma,
                'subsample': config.subsample,
                'colsample_bytree': config.colsample_bytree,
                'lambda': config.lambda_value,
                'alpha': config.alpha
            }
            model = xgb.train(
                params,
                dtrain,
                evals = [(deval, 'eval')],
                num_boost_round = config.num_boost_round,
                early_stopping_rounds=100
            )
            #model = lightgbm.train(params, train_data, valid_sets = [eval_data])
            
            eval_preds = model.predict(deval)
            del dtrain,deval
            gc.collect()
            
            fpr, tpr, thresholds = metrics.roc_curve(eval_y, eval_preds, pos_label = 1)
            eval_auc = metrics.auc(fpr, tpr)
            wandb.log({
                'validatoin_auc': eval_auc
            });
wandb.agent(sweep_id, function = train_sweep)

wandb: Agent Starting Run: 63ma531g with config:
wandb: 	alpha: 1.0279020505010756
wandb: 	colsample_bytree: 0.7517210944198072
wandb: 	gamma: 9
wandb: 	lambda_value: 0.08785536439422348
wandb: 	max_depth: 63
wandb: 	min_child_weight: 7
wandb: 	num_boost_round: 1503
wandb: 	subsample: 0.9297162741284126
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


[0]	eval-rmse:0.43342
[1]	eval-rmse:0.36955
[2]	eval-rmse:0.32718
[3]	eval-rmse:0.30072
[4]	eval-rmse:0.28151
[5]	eval-rmse:0.26845
[6]	eval-rmse:0.26110
[7]	eval-rmse:0.25586
[8]	eval-rmse:0.25117
[9]	eval-rmse:0.24814
[10]	eval-rmse:0.24567
[11]	eval-rmse:0.24390
[12]	eval-rmse:0.24229
[13]	eval-rmse:0.24176
[14]	eval-rmse:0.24080
[15]	eval-rmse:0.23947
[16]	eval-rmse:0.23950
[17]	eval-rmse:0.23952
[18]	eval-rmse:0.23952
[19]	eval-rmse:0.23950
[20]	eval-rmse:0.23949
[21]	eval-rmse:0.23951
[22]	eval-rmse:0.23955
[23]	eval-rmse:0.23951
[24]	eval-rmse:0.23953
[25]	eval-rmse:0.23955
[26]	eval-rmse:0.23957
[27]	eval-rmse:0.23953
[28]	eval-rmse:0.23951
[29]	eval-rmse:0.23951
[30]	eval-rmse:0.23954
[31]	eval-rmse:0.23956
[32]	eval-rmse:0.23958
[33]	eval-rmse:0.23956
[34]	eval-rmse:0.23954
[35]	eval-rmse:0.23954
[36]	eval-rmse:0.23952
[37]	eval-rmse:0.23954
[38]	eval-rmse:0.23953
[39]	eval-rmse:0.23951
[40]	eval-rmse:0.23952
[41]	eval-rmse:0.23950
[42]	eval-rmse:0.23946
[43]	eval-rmse:0.2394

[189]	eval-rmse:0.24826
[190]	eval-rmse:0.24827
[191]	eval-rmse:0.24827
[192]	eval-rmse:0.24827
[193]	eval-rmse:0.24827
[194]	eval-rmse:0.24826
[195]	eval-rmse:0.24826
[196]	eval-rmse:0.24826
[197]	eval-rmse:0.24826
[198]	eval-rmse:0.24826
[199]	eval-rmse:0.24827
[200]	eval-rmse:0.24825
[201]	eval-rmse:0.24824
[202]	eval-rmse:0.24827
[203]	eval-rmse:0.24827
[204]	eval-rmse:0.24827
[205]	eval-rmse:0.24826
[206]	eval-rmse:0.24826
[207]	eval-rmse:0.24827
[208]	eval-rmse:0.24826
[209]	eval-rmse:0.24826
[210]	eval-rmse:0.24828
[211]	eval-rmse:0.24827
[212]	eval-rmse:0.24826
[213]	eval-rmse:0.24826
[214]	eval-rmse:0.24826
[215]	eval-rmse:0.24825
[216]	eval-rmse:0.24825
[217]	eval-rmse:0.24825
[218]	eval-rmse:0.24826
[219]	eval-rmse:0.24825
[220]	eval-rmse:0.24825
[221]	eval-rmse:0.24826
[222]	eval-rmse:0.24826
[223]	eval-rmse:0.24823
[224]	eval-rmse:0.24823
[225]	eval-rmse:0.24822
[226]	eval-rmse:0.24823
[227]	eval-rmse:0.24823
[228]	eval-rmse:0.24823
[229]	eval-rmse:0.24825
[230]	eval-rmse:

[98]	eval-rmse:0.29654
[99]	eval-rmse:0.29653
[100]	eval-rmse:0.29652
[101]	eval-rmse:0.29653
[102]	eval-rmse:0.29654
[103]	eval-rmse:0.29654
[104]	eval-rmse:0.29654
[105]	eval-rmse:0.29655
[106]	eval-rmse:0.29656
[107]	eval-rmse:0.29655
[108]	eval-rmse:0.29655
[109]	eval-rmse:0.29655
[110]	eval-rmse:0.29654
[111]	eval-rmse:0.29654
[112]	eval-rmse:0.29654
[113]	eval-rmse:0.29655
[114]	eval-rmse:0.29654
[115]	eval-rmse:0.29654
[116]	eval-rmse:0.29653
[117]	eval-rmse:0.29654
[118]	eval-rmse:0.29654
[119]	eval-rmse:0.29655
[120]	eval-rmse:0.29656
[121]	eval-rmse:0.29656
[122]	eval-rmse:0.29657
[123]	eval-rmse:0.29657
[124]	eval-rmse:0.29657
[125]	eval-rmse:0.29657
[126]	eval-rmse:0.29657
[127]	eval-rmse:0.29657
[128]	eval-rmse:0.29655
[129]	eval-rmse:0.29655
[130]	eval-rmse:0.29654
[131]	eval-rmse:0.29655
[132]	eval-rmse:0.29654
[133]	eval-rmse:0.29656
[134]	eval-rmse:0.29656
[135]	eval-rmse:0.29654
[136]	eval-rmse:0.29653
[137]	eval-rmse:0.29655
[138]	eval-rmse:0.29655
[139]	eval-rmse:0.

validatoin_auc,█▅▇▁▆
validatoin_auc,0.97757


wandb: Agent Starting Run: cwlsgl31 with config:
wandb: 	alpha: 0.9055530101651356
wandb: 	colsample_bytree: 0.7015615991123816
wandb: 	gamma: 0
wandb: 	lambda_value: 0.9741609691592998
wandb: 	max_depth: 10
wandb: 	min_child_weight: 5
wandb: 	num_boost_round: 138
wandb: 	subsample: 0.9939909554651328
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


[0]	eval-rmse:0.42917
[1]	eval-rmse:0.36035
[2]	eval-rmse:0.31551
[3]	eval-rmse:0.28614
[4]	eval-rmse:0.26559
[5]	eval-rmse:0.25325
[6]	eval-rmse:0.24349
[7]	eval-rmse:0.23702
[8]	eval-rmse:0.23183
[9]	eval-rmse:0.22838
[10]	eval-rmse:0.22632
[11]	eval-rmse:0.22423
[12]	eval-rmse:0.22308
[13]	eval-rmse:0.22153
[14]	eval-rmse:0.22037
[15]	eval-rmse:0.21967
[16]	eval-rmse:0.21919
[17]	eval-rmse:0.21864
[18]	eval-rmse:0.21832
[19]	eval-rmse:0.21799
[20]	eval-rmse:0.21796
[21]	eval-rmse:0.21765
[22]	eval-rmse:0.21745
[23]	eval-rmse:0.21746
[24]	eval-rmse:0.21714
[25]	eval-rmse:0.21692
[26]	eval-rmse:0.21706
[27]	eval-rmse:0.21692
[28]	eval-rmse:0.21681
[29]	eval-rmse:0.21671
[30]	eval-rmse:0.21655
[31]	eval-rmse:0.21634
[32]	eval-rmse:0.21622
[33]	eval-rmse:0.21613
[34]	eval-rmse:0.21603
[35]	eval-rmse:0.21607
[36]	eval-rmse:0.21591
[37]	eval-rmse:0.21586
[38]	eval-rmse:0.21583
[39]	eval-rmse:0.21588
[40]	eval-rmse:0.21595
[41]	eval-rmse:0.21588
[42]	eval-rmse:0.21590
[43]	eval-rmse:0.2158

In [ ]:
wandb.finish()

### Best parameters

* colsample_bynode = 0.9669
* colsample_bytree = 0.9645
* lambda_l1 = 0.7217
* lambda_l2 = 3.207
* learning_rate = 0.09574
* max_bin = 184
* max_depth = 62
* min_data_in_leaf = 111
